# Bayesian Kintsugi Demo

Visualize model uncertainty as golden scars.

See `IMPLEMENTATION_PLAN.md` Section 1.5 for specification.

In [ ]:
# TODO: Implement kintsugi demo
# 1. Load trained VAE from results/vae_mnist.pt
# 2. Load test MNIST images
# 3. Apply OOD perturbations (occlusion, noise, rotation, mixing)
# 4. Run MC dropout sampling (n=50)
# 5. Render kintsugi overlays
# 6. Save gallery to results/kintsugi_gallery.png

raise NotImplementedError("Implement kintsugi demo")